In [3]:
# pip install urllib
# pip install m3u8
# pip install streamlink
from datetime import datetime, timedelta, timezone
import urllib
import m3u8
import streamlink
import cv2 #openCV
import time


In [4]:
#Ref: https://stackoverflow.com/questions/55631634/recording-youtube-live-stream-to-file-in-python
def get_stream(url):
    """
    Get upload chunk url
    input: youtube URL
    output: m3u8 object segment
    """
    
    #Try this line tries number of times, if it doesn't work, 
    # then show the exception on the last attempt
    # Credit, theherk, https://stackoverflow.com/questions/2083987/how-to-retry-after-exception
    tries = 10
    for i in range(tries):
        try:
            streams = streamlink.streams(url)
        except:
            if i < tries - 1: # i is zero indexed
                print(f"Attempt {i+1} of {tries}")
                time.sleep(0.1) #Wait half a second
                continue
            else:
                raise
        break
    #print(f"Stream choices: {streams.keys()})
    stream_url = streams["480p"] #Alternate, use "best"

    m3u8_obj = m3u8.load(stream_url.args['url'])
    return m3u8_obj.segments[0] #Parsed stream


def dl_stream(url, filename, chunks):
    """
    Download each chunk to file
    input: url, filename, and number of chunks (int)
    output: saves file at filename location
    returns none.
    """
    pre_time_stamp = datetime(1, 1, 1, 0, 0, tzinfo=timezone.utc)

    
    #Repeat for each chunk
    #Needs to be in chunks beceause 
    #  1) it's live and 
    #  2) it won't let you leave the stream open forever
    i=1
    while i <= chunks:
       
        #Open stream
        stream_segment = get_stream(url)
        
        #Get current time on video
        cur_time_stamp = stream_segment.program_date_time
        #Only get next time step, wait if it's not new yet
        if cur_time_stamp <= pre_time_stamp:
            #Don't increment counter until we have a new chunk
            print("NO   pre: ",pre_time_stamp, "curr:",cur_time_stamp)
            time.sleep(0.5) #Wait half a sec
            pass
        else:
            print("YES: pre: ",pre_time_stamp, "curr:",cur_time_stamp)
            print(f'#{i} at time {cur_time_stamp}')
            #Open file for writing stream
            file = open(filename, 'ab+') #ab+ means keep adding to file
            #Write stream to file
            with urllib.request.urlopen(stream_segment.uri) as response:
                html = response.read()
                file.write(html)
                
            #Update time stamp
            pre_time_stamp = cur_time_stamp
            time.sleep(stream_segment.duration) #Wait duration time - 1

            i += 1 #only increment if we got a new chunk

    return None

def openCVProcessing(saved_video_file):
    '''View saved video with openCV
    Add your other steps here'''
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        #
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically

#Do the work    
tempFile = "temp7.ts"  #files are format ts, open cv can view them
videoURL = "https://youtu.be/DoUOrTJbIu4" #Jackson hole town square, live stream

dl_stream(videoURL, tempFile, 10)


YES: pre:  0001-01-01 00:00:00+00:00 curr: 2021-05-19 15:59:34.840000+00:00
#1 at time 2021-05-19 15:59:34.840000+00:00
Attempt 1 of 10
Attempt 2 of 10
YES: pre:  2021-05-19 15:59:34.840000+00:00 curr: 2021-05-19 15:59:44.840000+00:00
#2 at time 2021-05-19 15:59:44.840000+00:00
YES: pre:  2021-05-19 15:59:44.840000+00:00 curr: 2021-05-19 15:59:49.840000+00:00
#3 at time 2021-05-19 15:59:49.840000+00:00
Attempt 1 of 10
YES: pre:  2021-05-19 15:59:49.840000+00:00 curr: 2021-05-19 15:59:59.839000+00:00
#4 at time 2021-05-19 15:59:59.839000+00:00
Attempt 1 of 10
YES: pre:  2021-05-19 15:59:59.839000+00:00 curr: 2021-05-19 16:00:04.839000+00:00
#5 at time 2021-05-19 16:00:04.839000+00:00
YES: pre:  2021-05-19 16:00:04.839000+00:00 curr: 2021-05-19 16:00:09.839000+00:00
#6 at time 2021-05-19 16:00:09.839000+00:00
YES: pre:  2021-05-19 16:00:09.839000+00:00 curr: 2021-05-19 16:00:19.839000+00:00
#7 at time 2021-05-19 16:00:19.839000+00:00
YES: pre:  2021-05-19 16:00:19.839000+00:00 curr: 2021

In [18]:
#Process the newly made file
openCVProcessing(tempFile)

NameError: name 'pre_time_stamp' is not defined

In [5]:
#Process newly saved video testing new things
def openCVProcessing(saved_video_file):
    #View saved video with openCV
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically


In [46]:
videoURL = "https://www.youtube.com/watch?v=DoUOrTJbIu4"
stream_segment = get_stream(videoURL)

# Attempt to combine streaming and working into one!

In [7]:
#Ref: https://stackoverflow.com/questions/55631634/recording-youtube-live-stream-to-file-in-python
def get_stream(url):
    """
    Get upload chunk url
    input: youtube URL
    output: m3u8 object segment
    """
    #Get stream
    streams = streamlink.streams(url)
    stream_url = streams["360p"] #Alternate, use "best"

    m3u8_obj = m3u8.load(stream_url.args['url'])
    return m3u8_obj.segments[0] #Parsed stream

def openCVProcessing(saved_video_file):
    '''View saved video with openCV
    Add your other steps here'''
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        #
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically

def dl_stream(url, filename, chunks):
    """
    Download each chunk to file
    input: url, filename, and number of chunks (int)
    output: saves file at filename location
    returns none.
    """
    pre_time_stamp = 0
    
    #Repeat for each chunk
    #Needs to be in chunks beceause 
    #  1) it's live and 
    #  2) it won't let you leave the stream open forever
    i=1
    while i <= chunks:
       
        #Open stream
        stream_segment = get_stream(url)
        #Get current time
        cur_time_stamp = \
            stream_segment.program_date_time.strftime("%Y%m%d-%H%M%S")
        #Only get next time step, wait if it's not new yet
        if pre_time_stamp == cur_time_stamp:
            #Don't increment counter if we didn't get a chunk
            pass
        else:
            print(f'#{i} at time {cur_time_stamp}')
            #Open file for writing stream
            file = open(filename, 'ab+') #ab+ means keep adding to file
            #Write stream to file
            with urllib.request.urlopen(stream_segment.uri) as response:
                html = response.read()
                openCVProcessing(html)
                #file.write(html)
            #Update time stamp
            pre_time_stamp = cur_time_stamp
            i += 1 #only increment if we got a new chunk

    return None


In [ ]:
tempFile = "temp.ts"  #files are format ts, open cv can view them
videoURL = "https://www.youtube.com/watch?v=DoUOrTJbIu4" #Jackson hole town square, live stream

dl_stream(videoURL, tempFile, 5)
#openCVProcessing(tempFile)

In [8]:
#Sandbox/scratch area

url = "https://www.youtube.com/watch?v=DoUOrTJbIu4" #Jackson hole town square, live stream

stream_segment = get_stream(url)
#Open file for writing stream
file = open("test.ts", 'ab+') #ab+ means keep adding to file
#Write stream to file
with urllib.request.urlopen(stream_segment.uri) as response:
    html = response.read()
    #openCVProcessing(html)
    file.write(html)
    openCVProcessing("test.ts")

PluginError: Unable to open URL: https://youtube.com/get_video_info (404 Client Error: Not Found for url: https://www.youtube.com/get_video_info?video_id=DoUOrTJbIu4&el=detailpage)

Scratch area

In [ ]:
stream_segment.

In [2]:
url = "https://youtu.be/DoUOrTJbIu4" #Jackson hole town square, live stream
stream_segment5 = get_stream(url)


NameError: name 'get_stream' is not defined

In [23]:
t1 = stream_segment.program_date_time
t2 = stream_segment2.program_date_time
t3 = stream_segment4.program_date_time
t5 = stream_segment5.program_date_time


In [45]:

durationDateTime = timedelta(seconds=stream_segment4.duration)
print(t3+durationDateTime+durationDateTime)
print(t5)

2021-05-18 20:47:49.848000+00:00
2021-05-18 20:47:49.847000+00:00


In [10]:
tries = 10
url = "https://youtu.be/DoUOrTJbIu4" #Jackson hole town square, live stream

for i in range(tries):
    try:
        streams = streamlink.streams(url)
    except:
        if i < tries - 1: # i is zero indexed
            print(f"Attempt {i+1} of {tries}")
            time.sleep(0.1) #Wait half a second
            continue
        else:
            raise
    break

stream_url = streams["360p"] #Alternate, use "best"

Attempt 1 of 10
Attempt 2 of 10


In [13]:
streams.keys()

odict_keys(['144p', '240p', '360p', '480p', '720p', '1080p', 'worst', 'best'])

In [36]:
help(stream_segment)

Help on Segment in module m3u8.model object:

class Segment(m3u8.mixins.BasePathMixin)
 |  Segment(uri=None, base_uri=None, program_date_time=None, current_program_date_time=None, duration=None, title=None, byterange=None, cue_out=False, cue_out_start=False, cue_in=False, discontinuity=False, key=None, scte35=None, scte35_duration=None, keyobject=None, parts=None, init_section=None, dateranges=None, gap_tag=None)
 |  
 |  A video segment from a M3U8 playlist
 |  
 |  `uri`
 |    a string with the segment uri
 |  
 |  `title`
 |    title attribute from EXTINF parameter
 |  
 |  `program_date_time`
 |    Returns the EXT-X-PROGRAM-DATE-TIME as a datetime. This field is only set
 |    if EXT-X-PROGRAM-DATE-TIME exists for this segment
 |    http://tools.ietf.org/html/draft-pantos-http-live-streaming-07#section-3.3.5
 |  
 |  `current_program_date_time`
 |    Returns a datetime of this segment, either the value of `program_date_time`
 |    when EXT-X-PROGRAM-DATE-TIME is set or a calculated

In [47]:
import inspect

inspect.getmembers(stream_segment, lambda a:not(inspect.isroutine(a)))


[('__class__', m3u8.model.Segment),
 ('__delattr__',
  <method-wrapper '__delattr__' of Segment object at 0x000002A10E0A8F28>),
 ('__dict__',
  {'uri': 'https://r5---sn-q4fl6nsy.googlevideo.com/videoplayback/id/DoUOrTJbIu4.0/itag/93/source/yt_live_broadcast/expire/1621371952/ei/0NejYNH-Hpupir4Pwei-wAM/ip/75.60.121.193/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D134/hls_chunk_host/r5---sn-q4fl6nsy.googlevideo.com/playlist_duration/30/manifest_duration/30/vprv/1/playlist_type/DVR/initcwndbps/9360/mh/9I/mm/44/mn/sn-q4fl6nsy/ms/lva/mv/m/mvi/5/pl/24/keepalive/yes/fexp/24001373,24007246/mt/1621349559/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,playlist_duration,manifest_duration,vprv,playlist_type/sig/AOq0QJ8wRgIhAKvALUF27ZSiGhHdsISTaFFF7A2-Sa6rs6Ef4JoZgzIEAiEAk0bWcYmWedj_CxSMANrIX2c8tLlmT9TX9oUhmbotQZY%3D/lsparams/hls_chunk_host,initcwndbps,mh,mm,mn,ms,mv,mvi,pl/lsig/AG3C_xAwRQIgOO3HTCLhz3zUeuUHEq-Qt6XuOMRAVzpDCLH